# Lavaanの独立モデル

$\chi^2$検定における独立モデル（Baseline Model）の自由度がsemopyとlavaanで一致しないことがある。

どういうことなのか？

## （確認）$\chi^2$検定と自由度

最尤法で母数を推定した場合、自由度

$$
df = \frac{1}{2} p (p + 1) - q
$$

の$\chi^2$分布に近似的に従う統計量が得られるので、これを用いて検定を行うことができる（豊田 2012）

ここで$p$は観測変数の数、$q$は自由母数の数

$\chi^2$検定自体はサンプルサイズへの依存性が高くて不正確だが、CFIなど他の指標の算出に用いられる。


### semopyの自由度の計算


Pythonのsemopyパッケージでの自由度の推定方法も豊田同様

```python
def calc_dof(model):
    p = len(model.vars['observed'])
    return p * (p + 1) // 2 - len(model.param_vals)
```

ref.: https://gitlab.com/georgy.m/semopy/-/blob/fa9f3e0bd2bdacac377c8ee339255b27f4ca22f8/semopy/stats.py#L187

### lavaanの自由度の推定

Rの{lavaan}パッケージでの自由度の推定。こちらも概ね同様な様子

[ソースコード](https://github.com/yrosseel/lavaan/blob/6f047c800206d23f246d484b9522295257614222/R/lav_partable_utils.R#L242-L256)を見ると

```R
# degrees of freedom
df <- ndat - npar
```

`npar`は（自由母数の数 ["total number of free parameters"](https://github.com/yrosseel/lavaan/blob/6f047c800206d23f246d484b9522295257614222/R/lav_partable_utils.R#L234-L235)）

`ndat`は（ブロックごとの標本母数の数 ["number of sample statistics per block"](https://github.com/yrosseel/lavaan/blob/6f047c800206d23f246d484b9522295257614222/R/lav_partable_utils.R#L109-L110)）

つまり

$$
標本母数 - 自由母数
$$

とのこと。標本母数のコードは複雑すぎてよくわからなかったが、いくつかのモデルで確認した感じでは基本的に $p (p + 1) / 2$ と一致する

## 独立モデル

**独立モデル（independent model, baseline model, null model)** は、すべての観測変数間に一切のパスを引かず、各変数の分散のみを自由推定するモデルのこと

> すべての観測変数間に一切のパスを引かず、各変数の分散のみを自由推定するモデルを独立モデルと考えることが一般的です。 （豊田 2014, p.193）

### semopyでの確認方法

`get_baseline_model()`で具体的なモデルの状況を確認することができる

```python
from semopy.stats import get_baseline_model
baseline = get_baseline_model(model)
baseline.inspect()
```

In [12]:
import statsmodels.api as sm
trees = sm.datasets.get_rdataset("trees", "datasets").data

import semopy
desc = """
    Volume ~ Height + Girth
    Height ~~ Height
    Girth ~~ Girth
    Height ~~ Girth
"""
model = semopy.Model(desc)
model.fit(trees)

from semopy.stats import get_baseline_model
baseline = get_baseline_model(model)
baseline.inspect()

,lval,op,rval,Estimate,Std. Err,z-value,p-value
0,Height,~~,Height,39.290323,9.979752,3.937004,0.000083
1,Girth,~~,Girth,9.530239,2.420683,3.937004,0.000083
2,Volume,~~,Volume,130.743288,33.208828,3.937004,0.000083


### lavaanでの確認方法

独立モデルは`lav_partable_independence()`（[source](https://github.com/yrosseel/lavaan/blob/a271f07a212fcf2902352833a37bde2202eb6b00/R/lav_partable_unrestricted.R#L34)）で取得できる

```r
lav <- lav_partable_independence(fit)
data.frame(lav)
```

また、`lav_partable_npar()`で自由母数の数を調べることができる

```r
# 自由母数の数
lav_partable_npar(lav)
```


```r
> library(lavaan)
> model <- 'Volume ~ Height + Girth'
> fit <- sem(model, data = trees, fixed.x=FALSE)
> lav <- lav_partable_independence(fit)
> data.frame(lav) |> dplyr::filter(free > 0L)
  id    lhs op    rhs user block group free     ustart exo
1  1 Volume ~~ Volume    1     1     1    1 261.486576   0
2  2 Height ~~ Height    1     1     1    2  39.290323   0
3  3  Girth ~~  Girth    1     1     1    3   9.530239   0
4  4 Height ~~  Girth    1     1     1    4         NA   0
```

## 両者の差異

lavaanにて、豊田（2014）のコードを実行したとき、独立モデルにて共分散も自由母数となっていた。

豊田のコードは`fixed.x=FALSE`という引数を指定している

```r
> fit4_P <- sem(model4_PLS, data=data4, fixed.x=F)
> lav <- lav_partable_independence(fit4_P)
> data.frame(lav)
   id lhs op rhs user block group free    ustart exo
1   1  x5 ~~  x5    1     1     1    1 1.9928182   0
2   2  x6 ~~  x6    1     1     1    2 1.7437518   0
3   3  x7 ~~  x7    1     1     1    3 0.8440103   0
4   4  x1 ~~  x1    1     1     1    4 1.0494829   0
5   5  x2 ~~  x2    1     1     1    5 0.8159293   0
6   6  x3 ~~  x3    1     1     1    6 1.4174088   0
7   7  x4 ~~  x4    1     1     1    7 1.2734846   0
8   8  x1 ~~  x2    1     1     1    8        NA   0
9   9  x1 ~~  x3    1     1     1    9        NA   0
10 10  x1 ~~  x4    1     1     1   10        NA   0
11 11  x2 ~~  x3    1     1     1   11        NA   0
12 12  x2 ~~  x4    1     1     1   12        NA   0
13 13  x3 ~~  x4    1     1     1   13        NA   0
```




#### `fixed.x`とは

[R: lavaan Options](https://search.r-project.org/CRAN/refmans/lavaan/html/lavOptions.html)を見ると

> fixed.x:
>
> If TRUE, the exogenous ‘x’ covariates are considered fixed variables and the means, variances and covariances of these variables are fixed to their sample values. If FALSE, they are considered random, and the means, variances and covariances are free parameters. If "default", the value is set depending on the mimic option.

`fixed.x=TRUE`だと外生変数は平均・分散・共分散がサンプルの値に固定される。`FALSE`だと平均・分散・共分散が自由母数として扱われる。デフォルトだと`mimic`のオプション（こちらのデフォルトは`mimic="lavaan"`）に依存する

**→ User Modelのほうで共分散を自由母数にするように指定していたため独立モデルもそれに引っ張られて共分散を自由母数にしている様子**



`fixed.x = F`にするのではなく自分で分散共分散も指定すれば共分散は自由母数にならず、semopyと一致する

```r
# fixed.x = Fにするのではなく自分で分散共分散も指定
model4_PLS_2 <- '
  f1 ~ x1 + x2 + x3 + x4
  f2 =~ x5 + x6 + x7
  f1 =~ f2
  f1 ~~ 0*f1
  f2 ~~ f2
  x1 ~~ x1
  x2 ~~ x2
  x3 ~~ x3
  x4 ~~ x4
  x1 ~~ x2
  x1 ~~ x3
  x1 ~~ x4
  x2 ~~ x3
  x2 ~~ x4
  x3 ~~ x4
'
fit4_P <- sem(model4_PLS_2, data=data4, fixed.x=T)
summary(fit4_P, standardized=T, fit.measures=T)

lav <- lav_partable_independence(fit4_P)
```

```r
> data.frame(lav)
  id lhs op rhs user block group free    ustart exo
1  1  x5 ~~  x5    1     1     1    1 1.9928182   0
2  2  x6 ~~  x6    1     1     1    2 1.7437518   0
3  3  x7 ~~  x7    1     1     1    3 0.8440103   0
4  4  x1 ~~  x1    1     1     1    4 1.0494829   0
5  5  x2 ~~  x2    1     1     1    5 0.8159293   0
6  6  x3 ~~  x3    1     1     1    6 1.4174088   0
7  7  x4 ~~  x4    1     1     1    7 1.2734846   0
```



### 外生変数の共分散も推定するBaseline Modelの定義もある

> a model which includes the means and variances of all observed variables plus the covariances of all observed exogenous variables. 
>
> [What are the saturated and baseline models in sem? | Stata FAQ](https://stats.oarc.ucla.edu/stata/faq/what-are-the-saturated-and-baseline-models-in-sem/)


`fixed.x=F`なので外生変数の分散や共分散も推定する → なのでBaseline Modelに入る、ということ？

### 他のBaseline Modelの説明

baseline modelsには2つのモデルが含まれる

1. unrestricted model: saturated model
2. independence model: モデルの変数の平均と分散をfitする。共分散は0に固定される


> The baseline models are an unrestricted model and an independence model.

> The unrestricted model is a fully saturated model, which fits all means, variances, and covariances of the specified Model Variables without imposing any structure on the data.

> The independence model fits all means and variances of the specified Model Variables. All covariances among the specified Model Variables are fixed to zero, which leads to a highly restrictive model.

[Overview of Structural Equation Models](https://www.jmp.com/support/help/en/17.2/index.shtml#page/jmp/overview-of-structural-equation-models.shtml)

### UCLAのLavaanのセミナーにおける説明

[Introduction to Structural Equation Modeling (SEM) in R with lavaan](https://stats.oarc.ucla.edu/r/seminars/rsem/#s3d)

> The baseline model can be thought of as the “worst-fitting” model and simply assumes that there is absolutely no covariance between variables. 

> essentially estimating only the variances. Since there is are no regression paths, there are no endogenous variables in our model

変数間の共分散がまったくなく、パスがないため分散だけを推定する、という豊田と同じ定義




### MplusにおけるBaseline Model

[What is the baseline model in Mplus? | Mplus FAQ](https://stats.oarc.ucla.edu/mplus/faq/what-is-the-baseline-model-in-mplus/)

> The baseline model Mplus prints the chi-square statistic for is a model where all of the structural (regression) paths are assumed to be zero (i.e., a null model). For the measurement model, all measurement paths from the latent variable to the observed indicator are 1, and the variance of the latent variable is 0 (since setting the path coefficients to one makes the latent variable just a linear composite of the indicator variables). Only the intercepts and residual variances of the observed endogenous (dependent) variables (i.e., read, math, science, and socst ) are estimated. 

- 構造方程式のパス係数はゼロと仮定（Null model）
- 測定方程式では因子負荷量が1で潜在変数の分散は0と仮定
- 観測された内生変数の切片と残差分散のみが推定される


## 関連するlavaanのメーリングリストのスレッド


:::{card} [Why independence model handle covariances as free parameters when fixed.x = FALSE ?](https://groups.google.com/g/lavaan/c/42AZhlDq9Ko)

- `fixed.x = T` でも独立モデルの共分散は自由母数ではないだけで、標本共分散を固定母数とするよ（＝共分散は0じゃないよ）
    - → （考察）なのでLavaanでは独立モデルの共分散を自由母数にする設定がデフォ
- 関連する論文があるよ：[Widaman, K. F., & Thompson, J. S. (2003). On specifying the null model for incremental fit indices in structural equation modeling. Psychological methods, 8(1), 16.](https://psycnet.apa.org/record/2003-00272-005)

:::


:::{card}
[Degrees of Freedom Calculations](https://groups.google.com/g/lavaan/c/zqjDtUx_b9c)

> Ed Rigdon has proposed an even more conservative option that fixes the covariances to the average covariance.

として、[Rigdon, E. E. (1998). The equal correlation baseline model for comparative fit assessment in structural equation modeling.](https://www.tandfonline.com/doi/abs/10.1080/10705519809540089)が紹介されている。

議論もあったが、より保守的なので採用したっぽい

> The choice is just a matter of convention but the consensus was that this practice inflates comparative fit indices and that free means offered more conservative values. 

これは共分散を自由母数にするわけではないが、変数間の共分散を均一にするというアイデアらしい

>  It is better described as one that constrains all the covariances to be equal to one another.  Here is the reference.
> https://www.tandfonline.com/doi/abs/10.1080/10705519809540089
:::

:::{card}
[Null/baseline model](https://groups.google.com/g/lavaan/c/iX2usbOxLlg)

> For continuous data, the default null model in lavaan (and all other SEM software I am aware of) is the "independence" model, which constraints all covariances to zero, and only freely estimates means and variances.
:::

## 参考文献

- 豊田秀樹（2012）『因子分析入門』
- 豊田秀樹（2014）『共分散構造分析［Ｒ編］―構造方程式モデリング―』

### 例：semopyのbaseline models